In [ ]:
import requests
import time
from datetime import datetime

In [ ]:
pip install jupyternotify

In [ ]:
%load_ext jupyternotify

In [ ]:
pip install twilio

In [ ]:
import os
from twilio.rest import Client

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = 'SID_HERE'
auth_token = 'TOKEN_HERE'
client = Client(account_sid, auth_token)

In [ ]:
api_token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyX25hbWUiOiJiZGZjYzRjYy0xN2FkLTRiODMtYTg0Ni1hYmQ0YTE3OGFlNzAiLCJ1c2VyX2lkIjoiYmRmY2M0Y2MtMTdhZC00YjgzLWE4NDYtYWJkNGExNzhhZTcwIiwidXNlcl90eXBlIjoiQkVORUZJQ0lBUlkiLCJtb2JpbGVfbnVtYmVyIjo5ODAwNjc5MjIyLCJiZW5lZmljaWFyeV9yZWZlcmVuY2VfaWQiOjg2MjQ0NzA1MjY2MzkwLCJzZWNyZXRfa2V5IjoiYjVjYWIxNjctNzk3Ny00ZGYxLTgwMjctYTYzYWExNDRmMDRlIiwidWEiOiJNb3ppbGxhLzUuMCAoTWFjaW50b3NoOyBJbnRlbCBNYWMgT1MgWCAxMF8xNF82KSBBcHBsZVdlYktpdC81MzcuMzYgKEtIVE1MLCBsaWtlIEdlY2tvKSBDaHJvbWUvOTAuMC40NDMwLjkzIFNhZmFyaS81MzcuMzYiLCJkYXRlX21vZGlmaWVkIjoiMjAyMS0wNS0xM1QxMjo1NDowNy40MDJaIiwiaWF0IjoxNjIwOTEwNDQ3LCJleHAiOjE2MjA5MTEzNDd9.1097nXKT0UH5wK4QNZH59Y5-bVjDJRgUzCTADVH0yCo'

In [ ]:
# %%notify -m "Cell Complete"
while True:
    headers = {'Content-Type': 'application/json',
               'Authorization': 'Bearer {0}'.format(api_token),
               'origin': 'https://selfregistration.cowin.gov.in',
              'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
    api = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByDistrict?district_id=330&date={}".format(datetime.today().strftime('%d-%m-%Y'))
    should_break = False
    abc = requests.get(api, headers=headers)
    should_continue = True
    print(str(abc.status_code) + ' ' + datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    if abc.ok:
        center_ob = abc.json()
        for center in center_ob['centers']:
            for session in center['sessions']:
                if session['min_age_limit'] == 18:
                    print(str(session['vaccine'])+ '       ' + str(session['available_capacity']) + ' '+ session['date'])
                    if session['available_capacity'] > 0:
                        %notify -m "Slot is Open"
                        if session['vaccine'] == 'COVISHIELD' and session['available_capacity'] > 1:
                            message = client.messages \
                                .create(
                                     body='Slot is Open for COVISHIELD Time: {}'.format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")),
                                     from_='+14156838789',
                                     to='+9112345678'
                                 )
                            print(message.sid)
                            %notify -m "Slot is Open for COVISHIELD"
                            should_break = True
        if should_break:
            break
        time.sleep(5)
    else:
        for i in range(10):
            abc = requests.get(api, headers=headers)
            print(abc)
            if abc.ok:
                should_continue = True
                break
            else:
                should_continue = False
            time.sleep(5)
        if not should_continue:
            %notify -m "Response is 401"
            break

In [ ]:
'Slot is Open for COVISHIELD Time: {}'.format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

In [ ]:
pip install twilio